# 要約 
このJupyterノートブックは、Gemma-2 9bモデルを使用して、人間の好みに基づくチャットボットの応答を予測するタスクに取り組んでいます。コンペティションにおいて、参加者はどちらのモデルの応答が好まれるかを予測するための機械学習モデルを構築する必要があります。

主な手法として、ノートブックではLoRA（Low-Rank Adaptation）およびQLoRA（Quantized Low-Rank Adaptation）を用いてモデルをファインチューニングしています。これは、トレーニング中に元の重みを凍結し、LoRAアダプタの重みだけを更新することで、メモリを効率的に使用しつつ、優れたパフォーマンスを実現する手法です。QLoRAでは、モデルの重みを量子化して計算リソースを最小化し、高精度の計算を維持します。

使用しているライブラリには、`transformers`、`bitsandbytes`、および`peft`が含まれ、これらを用いてモデル設定、トークナイザー、データセットにアクセスし、トレーニングを実行します。評価指標としては、対数損失（log loss）と精度（accuracy）が用いられ、特にKaggleのルールに基づいてパフォーマンスが計測されます。

ノートブックは、Gemma-2モデルのトレーニングを1エポックで実施し、得られた結果は評価セットに対して0.9371のログロス、リーダーボード上で0.941に達しました。このデモには、100サンプルのデータセットを使用しており、全体の設定やパラメータはカスタマイズ可能です。トレーニング結果はTensorBoardで記録され、リアルタイムでメモリ使用量や損失、精度を観察できるようになっています。

---


# 用語概説 
以下は、示されたJupyter Notebookに基づいて、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説です。これは、一般的な定義や広く知られた用語を除き、特に実務経験の少ない初心者がつまづく可能性がある特定の用語に焦点を当てています。

### 専門用語の解説

1. **Gemma-2 9b**:
   - 特定の大規模な言語モデル（LLM）で、9bは9ビリオン（約90億）のパラメータを持つことを示しています。LLMは、特に自然言語処理に特化したモデルのことを指します。

2. **LB (Leader Board)**:
   - コンペティションにおけるリーダーボードのスコアを指します。LBスコアはモデルの性能を示し、他の参加者と比較するための指標です。

3. **ファインチューニング**:
   - 既存のモデルに対して、新しいデータセットで再トレーニングを行い、モデルのパフォーマンスを高めるプロセスです。特に特定のタスクやドメインにおける精度を上げるために利用されます。

4. **量子化 (Quantization)**:
   - モデルのパラメータを、通常の32ビット浮動小数点数から低精度な形式（例：8ビットや4ビット）に変換するプロセスです。これにより、モデルのサイズを小さくし、計算効率を高めます。

5. **LoRA (Low-Rank Adaptation)**:
   - モデルのトレーニングを柔軟で効率的にするためのテクニックで、重みの更新を低ランク行列で近似します。これにより、トレーニングのメモリ使用量を下げながら性能を保ちます。

6. **QLoRA (Quantized LoRA)**:
   - LoRA技術を量子化技術と組み合わせたものです。これは、トレーニング中にモデル重みを量子化しつつ、計算は高精度で行うことで、リソースの使用効率を向上させます。

7. **トレーニング引数 (Training Arguments)**:
   - 機械学習モデルをトレーニングする際の設定値（例：エポック数やバッチサイズなど）をまとめたものです。これらの引数によって、トレーニングプロセスの挙動が決まります。

8. **メトリック (Metric)**:
   - モデルのパフォーマンスを評価するための指標です。具体例としては、精度（accuracy）、損失（loss）、ログロス（log loss）などが含まれます。

9. **TensorBoard**:
   - TensorFlowのための可視化ツールですが、PyTorchユーザーにも利用されています。トレーニング中のメトリックやモデルの挙動をグラフなどで可視化することが可能です。

10. **コールバック (Callback)**:
    - トレーニングの特定のイベント（例：エポック終了時）に基づいて自動的に呼び出される関数やメソッドです。モデルのトレーニング途中での様々な処理を実装するために役立ちます。

これらの用語は、Notebook内での実装や概念に特有なものであり、初心者が理解する上で特に重要です。

---


## このノートブックについて
このノートブックでは、Gemma-2 9bをトレーニングしてLB: 0.941を取得する方法を示します。推論コードは[こちら](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora)で見つけることができます。
私は、unslothチームがアップロードした4ビット量子化された[Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit)をベースモデルとして使用し、LoRAアダプタを追加して1エポックでトレーニングしました。

## 結果

評価セットとして `id % 5 == 0` を使用し、残りをすべてトレーニングに使用しました。

| サブセット | ログロス |
| - | - |
| 評価 | 0.9371 |
| LB | 0.941 |

## QLoRAファインチューニングとは？

従来のファインチューニングでは、重み ($\mathbf{W}$) の更新が以下のように行われます：

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

ここで、$L$ はこのステップでの損失値、$\eta$ は学習率です。

[LoRA](https://arxiv.org/abs/2106.09685)は、$\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$ を、$r \ll \text{min}(\text{d}, \text{k})$ の2つの (はるかに) 小さい行列、$\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$ と$\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$ に因子分解して近似しようとします。

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

トレーニング中、元の重みは凍結されるため、$\mathbf{A}$ と $\mathbf{B}$ のみが更新されます。これにより、トレーニング中に更新する必要のある元の重みの割合はごくわずか (例えば <1%) に抑えられます。この方法で、トレーニング中のGPUメモリ使用量を大幅に削減しながら、通常の (フル) ファインチューニングと同等のパフォーマンスを達成できます。

[QLoRA](https://arxiv.org/abs/2305.14314)は、LLMの量子化により効率をさらに向上させます。例えば、8Bパラメータモデルは32ビットで32GBのVRAMを必要としますが、量子化された8ビット/4ビット8Bモデルはそれぞれ8GB/4GBで済みます。
QLoRAは、低精度 (例えば8ビット) でLLMの重みを量子化する一方で、フォワード/バックワードの計算は高精度 (例えば16ビット) で行い、LoRAアダプタの重みも高精度で保持されていることに注意してください。

A6000を使用した1エポックは、4ビットで約15時間、8ビットで約24時間かかり、ログロスの差は大きくありませんでした。

## 注意
Kaggleカーネルでのフルトレーニングには非常に長い時間がかかります。フルトレーニングを実行するには外部の計算リソースを使用することをお勧めします。
このノートブックではデモ目的で100サンプルのみを使用していますが、その他はすべて私の設定と同じです。


In [ ]:
# gemma-2はtransformers>=4.42.3から利用可能です
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [ ]:
import os
import copy
from dataclasses import dataclass
# 必要なライブラリをインポート
from torch.utils.tensorboard import SummaryWriter
import psutil  # システムとプロセスのリソース使用情報を取得するためのライブラリ
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    TrainerCallback,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

In [ ]:
# !pip install --upgrade kaggle
# # まず、これを実行してから作成する必要があります
# !kaggle datasets list

In [ ]:
# import json

# token = {
#     "username": "qinhaoyang",
#     "key": "02c6cdf132dabb5ddd9de0d37d8a7777"
# }

# with open('/root/.kaggle/kaggle.json', 'w') as file:
#     json.dump(token, file)
# with open('/kaggle/working/kaggle.json', 'w') as file:
#     json.dump(token, file)

In [ ]:
# !chmod 600 /root/.kaggle/kaggle.json

# # JSONデータを定義
# data ={
#   "title": "LMSYS-model", # データセットのタイトル
#   "subtitle": "",
#   "description": "",
#   "id": "qinhaoyang/LMSYS-model",
#   "licenses": [
#         {
#             "name": "unknown"
#         }
#     ],
#     "keywords": [],
#     "collaborators": [],
#     "data": []
# }


# with open('/kaggle/working/dataset-metadata.json', 'w') as file:
#     json.dump(data, file)

### 設定


In [ ]:
#!kaggle kernels output qinhaoyang/training-gemma-2-9b-4-bit-qlora-fine-tuning -p /kaggle/working

In [ ]:
#!kaggle datasets version -p /kaggle/working/output/ -m "データセットの変更内容についての説明" --dir-mode tar

In [ ]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4ビット量子化されたgemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 4
    gradient_accumulation_steps: int = 1  # グローバルバッチサイズは8
    per_device_eval_batch_size: int = 4
    n_epochs: int = 1
    freeze_layers: int = 16  # 合計42層があるため、最初の16層にはアダプタを追加しない
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

#### トレーニング引数


In [ ]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=200,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

#### LoRA設定


In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # 自自己注意メカニズムのみをターゲットする
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

### トークナイザーとモデルのインスタンス化


In [ ]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # <eos>を末尾に追加
tokenizer.padding_side = "right"

In [ ]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False  # キャッシュの使用を無効にする
model = prepare_model_for_kbit_training(model)  # 低ビットでのトレーニングの準備
model = get_peft_model(model, lora_config)  # LoRAモデルの取得
model

In [ ]:
model.print_trainable_parameters()  # トレーニング可能なパラメータを表示

### データセットのインスタンス化


In [ ]:
# ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
ds = Dataset.from_csv("/kaggle/input/lmsys-72k-dataset/lmsys-7.2k.csv")  # データセットの読み込み
# ds = ds.select(torch.arange(100))  # デモ目的で最初の100データを使用

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]  # プロンプトを処理
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aを処理
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bを処理
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # テキストを組み合わせる
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # トークナイザーでトークン化
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0  # モデルAが勝った場合
            elif b_win:
                label = 1  # モデルBが勝った場合
            else:
                label = 2  # 引き分けの場合
            labels.append(label)  # ラベルを追加
        return {**tokenized, "labels": labels}  # トークン化された結果とラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))  # テキストを処理する

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンス化
ds = ds.map(encode, batched=True)  # データセットにトークナイザーを適用

### メトリックの計算

LBで使用されるログロスと精度を補助的なメトリックとして計算します。


In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # 予測結果を取得
    labels = eval_preds.label_ids  # ラベルを取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # 予測値を確率に変換
    loss = log_loss(y_true=labels, y_pred=probs)  # ログロスを計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度を計算
    return {"acc": acc, "log_loss": loss}  # 精度とログロスを返す

### 分割

ここでは、`id % 5` に従ってトレーニングと評価を分割します。


In [ ]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニング用
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 評価用
    ) 
    for fold_idx in range(config.n_splits)
]

In [ ]:
# チェックポイントをロード
checkpoint = "/kaggle/input/lmsys-gemma-checkpoint/output/checkpoint-10200"

In [ ]:
#%tensorboard --logdir=/kaggle/working

In [ ]:
# 現在のプロセスのメモリ使用状況を記録し、TensorBoardに書き込む関数を定義します
def log_memory_usage(step, writer):  # step は記録のステップ識別子、writer はSummaryWriterのインスタンス
    # 現在のプロセスを取得
    process = psutil.Process(os.getpid())
    # プロセスのメモリ情報を取得
    mem_info = process.memory_info()
    # RSS（常駐セットサイズ）とVMS（仮想メモリサイズ）をMBに変換し、TensorBoardに記録
    writer.add_scalar('Memory Usage/RSS (MB)', mem_info.rss / (1024 * 1024), step)  # 物理メモリ使用量
    writer.add_scalar('Memory Usage/VMS (MB)', mem_info.vms / (1024 * 1024), step)  # 仮想メモリ使用量
    print(f"メモリ使用量がステップ {step} で記録されました")

# 各エポック終了時にメモリ使用状況を記録するカスタムのTrainerCallbackサブクラスを作成
class MemoryUsageLoggingCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):  # 各エポック終了時に呼び出されるコールバックメソッド
        # 現在のエポック数を記録のステップとして使用
        current_epoch = state.epoch
        # 前に定義した関数を呼び出し、メモリ使用状況を記録
        log_memory_usage(current_epoch, tb_writer)
    
    def on_log(self, args, state, control, **kwargs):
        logs = kwargs.get("logs", {})
        for key, value in logs.items():
            if isinstance(value, (int, float)):
                tb_writer.add_scalar(f"{key.capitalize()}", value, state.global_step)
                print(f"{key}: {value}")

        # トレーニングと評価の損失と精度も追加で記録
        if "loss" in logs:
            tb_writer.add_scalar("Loss/train", logs["loss"], state.global_step)
        if "eval_loss" in logs:
            tb_writer.add_scalar("Loss/eval", logs["eval_loss"], state.global_step)
        if "accuracy" in logs:
            tb_writer.add_scalar("Accuracy/train", logs["accuracy"], state.global_step)
        if "eval_accuracy" in logs:
            tb_writer.add_scalar("Accuracy/eval", logs["eval_accuracy"], state.global_step)

# SummaryWriterを初期化
tb_writer = SummaryWriter(log_dir="/kaggle/working/Gemma/tensorboard_logs")

# トレーニングと評価のデータセットを定義
train_idx, eval_idx = folds[config.fold_idx]

# Trainerを初期化し、モデル、データセットなどの設定を含め、メモリ使用記録のカスタムコールバックも追加
trainer = Trainer(
    args=training_args,  # トレーニング引数
    model=model,  # トレーニングするモデル
    tokenizer=tokenizer,  # トークナイザー
    train_dataset=ds.select(train_idx),  # トレーニングデータセット
    eval_dataset=ds.select(eval_idx),  # 評価データセット
    compute_metrics=compute_metrics,  # 評価指標を計算するメソッド
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # データ処理
    callbacks=[MemoryUsageLoggingCallback()],  # Trainerにカスタムコールバックを追加
)

# チェックポイントがあるか確認してトレーニングを再開
if checkpoint:
    trainer.train(resume_from_checkpoint=checkpoint)
else:
    trainer.train()

# トレーニング後、SummaryWriterを閉じてリソースを解放
tb_writer.close()

In [ ]:
model.save_pretrained("/kaggle/working/Gemma") # モデルを保存するローカルパスを指定
tokenizer.save_pretrained("/kaggle/working/Gemma") # 必要に応じて、トークナイザーも同時に保存

In [ ]:
# !kaggle datasets version -p /kaggle/working -m "データセットの変更内容についての説明" --dir-mode tar